In [1]:
# imports

import pandas as pd
import numpy as np

In [2]:
#read csv files in

df2019 = pd.read_csv("NBA_data/NBA_Season_avgs_2019_20.csv")
df2018 = pd.read_csv("NBA_data/NBA_Season_avgs_2018_19.csv")
df2017 = pd.read_csv("NBA_data/NBA_Season_avgs_2017_18.csv")
df2016 = pd.read_csv("NBA_data/NBA_Season_avgs_2016_17.csv")
df2015 = pd.read_csv("NBA_data/NBA_Season_avgs_2015_16.csv")
df2014 = pd.read_csv("NBA_data/NBA_Season_avgs_2014_15.csv")

In [3]:
# add year columns to each

df2019['Year'] = "2019"
df2018['Year'] = "2018"
df2017['Year'] = "2017"
df2016['Year'] = "2016"
df2015['Year'] = "2015"
df2014['Year'] = "2014"

In [4]:
# combine files
final_df = pd.concat([df2019, df2018, df2017, df2016, df2015, df2014], axis=0)

In [5]:
final_df = final_df.dropna()
final_df["playoffs_y_n"] = final_df["Team"].map(lambda x: 1 if "*" in x else 0)
final_df["Team"] = final_df["Team"].str.replace("*", "")

final_df

,Rk,Team,G,W,L,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,playoffs_y_n
0,1.0,Dallas Mavericks,67.0,53.0,12.0,241.5,41.6,90.0,0.462,15.3,...,36.4,47.0,24.5,6.3,5.0,12.8,19.0,116.4,2019,0
1,2.0,Milwaukee Bucks,65.0,49.0,14.0,240.8,43.5,91.2,0.477,13.7,...,42.2,51.7,25.9,7.4,6.0,14.9,19.2,118.6,2019,1
2,3.0,Houston Rockets,64.0,44.0,20.0,241.2,41.1,90.7,0.454,15.4,...,34.6,44.9,21.5,8.5,5.1,14.7,21.6,118.1,2019,0
3,4.0,Portland Trail Blazers,66.0,46.0,18.0,240.8,41.9,90.9,0.461,12.6,...,35.4,45.5,20.2,6.1,6.2,13.0,21.4,113.6,2019,0
4,5.0,Atlanta Hawks,67.0,40.0,27.0,243.0,40.6,90.6,0.449,12.0,...,33.4,43.3,24.0,7.8,5.1,16.2,23.1,111.8,2019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26.0,Utah Jazz,82.0,25.0,57.0,240.3,35.4,79.2,0.447,7.4,...,31.9,44.0,19.9,7.6,6.0,15.3,19.3,95.1,2014,0
26,27.0,Miami Heat,82.0,21.0,61.0,240.6,35.2,77.2,0.456,6.8,...,30.0,39.1,19.8,7.8,4.5,14.8,20.0,94.7,2014,0
27,28.0,Charlotte Hornets,82.0,16.0,66.0,242.7,35.5,84.5,0.420,6.1,...,34.1,44.1,20.2,6.1,5.5,11.9,18.2,94.2,2014,0
28,29.0,Philadelphia 76ers,82.0,18.0,64.0,241.5,33.7,82.6,0.408,8.4,...,30.9,42.9,20.5,9.6,5.9,17.7,21.7,92.0,2014,0


In [10]:
final_df['MP'].describe()

count    180.000000
mean     241.711667
std        0.796590
min      240.300000
25%      241.200000
50%      241.800000
75%      242.300000
max      244.000000
Name: MP, dtype: float64

# Create a Train Test Split

In [6]:
final_df_model = final_df[final_df['Year'] != '2019']
final_df_predict = final_df[final_df['Year'] == '2019']
final_df_predict_X = final_df_predict.drop(columns=["Team", 'Rk', 'playoffs_y_n', 'Year'])

In [9]:
from sklearn.model_selection import train_test_split

y = final_df_model["playoffs_y_n"]
X = final_df_model.drop(columns=["Team", 'Rk', 'playoffs_y_n', 'Year'])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [12]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model

In [13]:
# Train the Model# Support vector machine linear classifier
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [14]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.7946428571428571
Testing Data Score: 0.7631578947368421


# Hyperparameter Tuning

Use `GridSearchCV` to tune model's parameters

In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model2, param_grid, verbose=3)

In [16]:
grid.fit(X_train_scaled, y_train)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.763, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.676, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.784, total=   0.0s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.763, total=   0.0s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.676, total=   0.0s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.784, total=   0.0s
[CV] C=1, gamma=0.01 .................................................
[CV] ............

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    0.1s finished
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001}
0.7857142857142857


# Predictions

In [26]:
predictions = model2.predict(final_df_predict_X)

In [27]:
len(final_df_predict_X)

30

In [28]:
len(final_df_predict["playoffs_y_n"])

30

In [29]:
pd.DataFrame({"Prediction": predictions, "Actual": final_df_predict["playoffs_y_n"]})

,Prediction,Actual
0,1,0
1,1,1
2,1,0
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


In [20]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model2.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.53      1.00      0.69        20

    accuracy                           0.53        38
   macro avg       0.26      0.50      0.34        38
weighted avg       0.28      0.53      0.36        38



/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
